<img src="images/ragna-logo.png" width=15% align="right"/>

# Set up an offline Large Language Model

<hr>

## What is a "local" or "offline" LLM

Large Language Models (LLMs) like OpenAI's GPT.

Most are proprietary, need access keys. Data privacy, vendor lock-in, cost savings.

Offline, local, or open weights LLMs can be self-hosted on local computers.

In our case, we're running it on a cloud platform, but each machine you have access to is essentially for individual. This allows us to have a standard tutorial environment with 1 T4 GPU.

## Mistral7BInstruct

Well performing and popular model for offline usage. Learn more at [mistral.ai](https://mistral.ai/news/announcing-mistral-7b/)

### Aside: Vocabulary

* Quantization
* Inference

## Exllamav2

TODO: Steps to run the LLM locally. Note, do not introduce RAG yet.

<hr>

**✨ Next: [TODO] →**

<hr>